In [14]:
import pandas as pd
import numpy as np
from pathlib import Path

data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')
#data_folder = Path('E:/Downloads/')
#data_folder = Path('C:/Users/Paul/Downloads/')
#train2_irf_0000_corr98.csv
df = pd.read_csv(data_folder/'train_redux2.csv', \
                 dtype='int8')
y = pd.read_csv(data_folder/'y2.csv', \
                sep=';', usecols=[1], dtype='int8')

print(df.shape, y.shape)

(26495, 18124) (26495, 1)


In [15]:
%%time
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, GaussianNoise, BatchNormalization
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt import space_eval
import warnings
from hyperopt.pyll.base import scope

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)

def Performance(Model,Y,X):
    AUC = roc_auc_score(Y, Model.predict_proba(X, verbose=0))
    print ('the AUC is : %0.4f' %  AUC)
    return AUC

def auc_model(params):
    clf = xgb.XGBClassifier(**params)
    clf.fit(train_x, train_y)
    classifier.fit(train_x, train_y, validation_data=(test_x, test_y), shuffle=True, \
               epochs=50, batch_size=100, callbacks=callbacks)
    AUC = Performance(clf, test_y, test_x)
    return AUC

def last_model(params):
    mlp = Sequential()
    mlp.add(Dense(units=256, kernel_initializer = 'lecun_uniform', \
                  activation='elu', input_dim = train_x.shape[1]))
    mlp.add(Dropout(0.25))
    mlp.add(Dense(units=256, kernel_initializer = 'lecun_uniform', activation='elu'))
    mlp.add(Dense(units=params['units3'], kernel_initializer = 'lecun_uniform', activation='elu'))
    mlp.add(Dropout(params['dropout3']))
    mlp.add(Dense(units = 1, kernel_initializer = 'lecun_uniform', activation = 'sigmoid'))
    mlp.compile(optimizer='adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])
    mlp.fit(train_x, train_y, validation_data=(test_x, test_y), shuffle=True, \
               epochs=50, batch_size=1024, callbacks=callbacks, verbose=0)
    auc = Performance(mlp, valid_y, valid_x)
    return mlp, auc

callbacks = [EarlyStopping(monitor='val_acc', patience=3, verbose=0)]
                #ModelCheckpoint(filepath='E:\\Downloads\\best_model_{epoch:02d}-{val_acc:.3f}.h5', \
                #monitor='val_acc', save_best_only=True)]

param_space = {
    'units3': hp.choice('units3', [64, 128]),
    'dropout3': hp.choice('dropout3', [0.4,0.5,0.6])
    }

def f(params):
    global best
    global best_params
    mlp = Sequential()
    mlp.add(Dense(units=256, kernel_initializer = 'lecun_uniform', \
                  activation='elu', input_dim = train_x.shape[1]))
    mlp.add(Dropout(0.25))
    mlp.add(Dense(units=256, kernel_initializer = 'lecun_uniform', activation='elu'))
    mlp.add(Dense(units=params['units3'], kernel_initializer = 'lecun_uniform', activation='elu'))
    mlp.add(Dropout(params['dropout3']))
    mlp.add(Dense(units = 1, kernel_initializer = 'lecun_uniform', activation = 'sigmoid'))
    mlp.compile(optimizer='adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])
    mlp.fit(train_x, train_y, validation_data=(test_x, test_y), shuffle=True, \
               epochs=50, batch_size=1024, callbacks=callbacks, verbose=0)
    auc = Performance(mlp, test_y, test_x)
    if auc > best:
        best = auc
        best_params = params
        print('*****', best, params)
    if auc > 0.83 and auc > (best - 0.004):
        print (params)
    return {'loss': -auc, 'status': STATUS_OK}


s1 = StratifiedShuffleSplit(n_splits=1, test_size=0.20)
for t1_index, val_index in s1.split(df, y):
    s1_x, s1_y = pd.DataFrame(df.iloc[t1_index]), y.iloc[t1_index].values.ravel()
    valid_x, valid_y = df.iloc[val_index], y.iloc[val_index].values.ravel()
 
#cv = StratifiedKFold(n_splits=4, shuffle=True)
cv = StratifiedShuffleSplit(n_splits=1, test_size=0.25)
for train_index, test_index in cv.split(s1_x, s1_y):
    train_x, train_y = pd.DataFrame(s1_x.iloc[train_index]), s1_y[train_index]
    test_x, test_y = pd.DataFrame(s1_x.iloc[test_index]), s1_y[test_index]

    trials = Trials()
    best = 0
    fmin(f, param_space, algo=tpe.suggest, max_evals=20, trials=trials)
    print ('\nbest:')
    model, AUC = last_model(best_params)
    print(best)
    output = 'MLP_' + str(int(AUC * 10000)) + '.pkl'
    #model.save(data_folder/output)
    #print(space_eval(param_space, best_params))
    print(best_params)
    print('\n')

the AUC is : 0.7068
***** 0.7067545215328255 {'dropout3': 0.6, 'units3': 64}
the AUC is : 0.8255
***** 0.8254998321899373 {'dropout3': 0.6, 'units3': 128}
the AUC is : 0.8313
***** 0.8313292410265053 {'dropout3': 0.4, 'units3': 128}
{'dropout3': 0.4, 'units3': 128}
the AUC is : 0.6921
the AUC is : 0.8309
{'dropout3': 0.6, 'units3': 64}
the AUC is : 0.8284
the AUC is : 0.8267
the AUC is : 0.8317
***** 0.8316514682004998 {'dropout3': 0.5, 'units3': 64}
{'dropout3': 0.5, 'units3': 64}
the AUC is : 0.6263
the AUC is : 0.8320
***** 0.8320222719716075 {'dropout3': 0.4, 'units3': 64}
{'dropout3': 0.4, 'units3': 64}
the AUC is : 0.8286
the AUC is : 0.8316
{'dropout3': 0.6, 'units3': 128}
the AUC is : 0.8312
{'dropout3': 0.5, 'units3': 128}
the AUC is : 0.8322
***** 0.8322107144191266 {'dropout3': 0.4, 'units3': 128}
{'dropout3': 0.4, 'units3': 128}
the AUC is : 0.7127
the AUC is : 0.8055
the AUC is : 0.8338
***** 0.8338361119726037 {'dropout3': 0.4, 'units3': 64}
{'dropout3': 0.4, 'units3': 64

In [5]:
from keras.models import load_model
#0.8245 ENS
#0.8468 (0.8496) search 100
classifier = load_model(data_folder/'MLP_8468.pkl') 
t1 = pd.read_csv('E:\\Downloads\\drop_corr98.csv', names=['a','b'])
t1.drop(['b'], axis=1, inplace=True)
t1 = t1.drop(t1.index[:1])
t2 = pd.read_csv('E:\\Downloads\\irf_weight_00000.csv', names=['a','b'])
t2.drop(['a'], axis=1, inplace=True)
t2.columns = ['a']
t2 = t2.drop(t2.index[:1])
concatenated = pd.concat([t1, t2], axis=0)
concatenated.drop_duplicates('a')
test = pd.read_csv(data_folder/'test_redux.csv')
mlp_test = test.drop(concatenated['a'].tolist(), axis=1)
probas_ = classifier.predict_proba(mlp_test)
print(classifier.get_config())
print(classifier.summary())

2163713
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_361 (Dense)            (None, 256)               2064896   
_________________________________________________________________
dropout_181 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_362 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_363 (Dense)            (None, 128)               32896     
_________________________________________________________________
dropout_182 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_364 (Dense)            (None, 1)                 129       
Total params: 2,163,713
Trainable params: 2,163,713
Non-trainable params: 0
__________________________________________________________

In [8]:
submission = pd.DataFrame(columns={'Ids', 'TARGET'})
submission['Ids'] = [i for i in range(26500,39750)]
submission['TARGET'] = probas_
submission = submission[['Ids', 'TARGET']]
submission.to_csv(data_folder/'submission.csv', sep=';', index=False, header={'Ids', 'TARGET'})